Feature Engineering (Emotion, Formality (POS), Number of Unique Token, Length of each post); Basic TF IDF vectoriser + Cosine Similarity - Valli & Run Lin

In [3]:
import pandas as pd
from afinn import Afinn
from textblob import TextBlob

# check which is used
import spacy, collections, readability
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk import ne_chunk, pos_tag, ngrams
from pickle import dump, load
from nltk.parse import CoreNLPParser


In [3]:
df = pd.read_csv('MBTI.csv')
df = df.drop(columns = ['Unnamed: 0'])
df

,Tag,Cleaned_Text,Cleaned_Text_No_Emoticon,Text_stemmed,Bigram,Reddit,Twitter,Typology,Kaggle
0,ENFJ,memeincorrect quote sure trying keep peace family,memeincorrect quote sure trying keep peace family,memeincorrect quot sure tri peac famili,memeincorrect quote sure trying keep_peace family,1,0,0,0
1,ENTP,memefour distinct flavors nt,memefour distinct flavors nt,memefour distinct flavor,memefour distinct flavors nt,1,0,0,0
2,ENTP,memethis definitely intp looks like,memethis definitely intp looks like,memethi definit intp look like,memethis definitely intp looks_like,1,0,0,0
3,ISFJ,theory questionwhat type admire,theory questionwhat type admire,theori questionwhat type admir,theory_questionwhat type admire,1,0,0,0
4,ISTJ,memeistj x enfp real,memeistj x enfp real,memeistj enfp real,memeistj x enfp real,1,0,0,0
...,...,...,...,...,...,...,...,...,...
134164,ISFP,always think cats fi doms reason websites beco...,always think cats fi doms reason websites beco...,think cat dom reason websit nazi perci nerd le...,always think cats fi_doms reason websites beco...,0,0,0,1
134165,ENFP,soif thread already exists someplace else heck...,soif thread already exists someplace else heck...,soif thread exist someplac heck delet hereooop...,soif thread already_exists someplace else heck...,0,0,0,1
134166,INTP,many questions things would take purple pill p...,many questions things would take purple pill p...,question thing purpl pill pick win lotteri num...,many questions things would take purple pill p...,0,0,0,1
134167,INFP,conflicted right comes wanting children honest...,conflicted right comes wanting children honest...,conflict right come want children honest mater...,conflicted right comes wanting children honest...,0,0,0,1


In [5]:
def polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

def subjectivity(text):
    blob = TextBlob(text)
    return blob.sentiment.subjectivity

polarity('Just get out, please. I hate it here.')

-0.8

In [228]:
features_df = df[['Tag', 'Cleaned_Text']]

afinn = Afinn(emoticons=True)
features_df['Afinn Score'] = [afinn.score(text) for text in df.Cleaned_Text]
features_df['Polarity'] = [polarity(text) for text in df.Cleaned_Text]
features_df['Subjectivity'] = [subjectivity(text) for text in df.Cleaned_Text]
features_df['Length'] = [len(text) for text in df.Cleaned_Text]

    
features_df

<ipython-input-228-e768e1fbfb73>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df['Afinn Score'] = [afinn.score(text) for text in df.Cleaned_Text]
<ipython-input-228-e768e1fbfb73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df['Polarity'] = [polarity(text) for text in df.Cleaned_Text]
<ipython-input-228-e768e1fbfb73>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Tag,Cleaned_Text,Afinn Score,Polarity,Subjectivity,Length
0,ENFJ,memeincorrect quote sure trying keep peace family,2.0,0.500000,0.888889,49
1,ENTP,memefour distinct flavors nt,0.0,0.300000,0.300000,28
2,ENTP,memethis definitely intp looks like,2.0,0.000000,0.500000,35
3,ISFJ,theory questionwhat type admire,3.0,0.000000,0.000000,31
4,ISTJ,memeistj x enfp real,0.0,0.200000,0.300000,20
...,...,...,...,...,...,...
134164,ISFP,always think cats fi doms reason websites beco...,23.0,0.139790,0.579605,2859
134165,ENFP,soif thread already exists someplace else heck...,144.0,0.201315,0.563804,4496
134166,INTP,many questions things would take purple pill p...,11.0,0.101508,0.535257,3469
134167,INFP,conflicted right comes wanting children honest...,54.0,0.095494,0.558648,5423


In [ ]:
features_df.to_csv(r'C:\Users\School\Desktop\4222\Project MBTI\Basic Features on Cleaned Data.csv', index = False)


In [2]:
raw_df = pd.read_excel('BT4222_Combined_dataset_Version 2.xlsx')
raw_df


,Tag,Text,Reddit,Twitter,Typology,Kaggle
0,INFP,Meme<3,1,0,0,0
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0
2,INTP,MemeENFP Avatar,1,0,0,0
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0
4,INTJ,StereotypesINFP 🦋,1,0,0,0
...,...,...,...,...,...,...
214114,ISFP,https://www.youtube.com/watch?v=t8edHB_h908|||...,0,0,0,1
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1
214116,INTP,'So many questions when i do these things. I ...,0,0,0,1
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1


In [3]:
# basic cleaning 
raw_df['Text'] = [str(text).replace(':', '',) for text in raw_df.Text]
raw_df['Text'] = [str(text).replace('/', '',) for text in raw_df.Text]
raw_df = raw_df[raw_df['Text']!= ""]

In [ ]:
# basic analysis
afinn = Afinn(emoticons=True)
raw_df['Afinn Score'] = [afinn.score(text) for text in raw_df.Text]

In [6]:
raw_df['Polarity'] = [polarity(text) for text in raw_df.Text]
raw_df['Subjectivity'] = [subjectivity(text) for text in raw_df.Text]
raw_df['Length'] = [len(text) for text in raw_df.Text]
raw_df

<ipython-input-6-14adb08c6a20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['Polarity'] = [polarity(text) for text in raw_df.Text]
<ipython-input-6-14adb08c6a20>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['Subjectivity'] = [subjectivity(text) for text in raw_df.Text]
<ipython-input-6-14adb08c6a20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Afinn Score,Polarity,Subjectivity,Length
0,INFP,Meme<3,1,0,0,0,3.0,0.000000,0.000000,7
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0,2.0,0.500000,0.888889,83
2,INTP,MemeENFP Avatar,1,0,0,0,0.0,0.000000,0.000000,16
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0,0.0,0.300000,0.300000,32
4,INTJ,StereotypesINFP 🦋,1,0,0,0,0.0,0.000000,0.000000,18
...,...,...,...,...,...,...,...,...,...,...
214114,ISFP,httpswww.youtube.comwatch?v=t8edHB_h908|||IxFP...,0,0,0,1,21.0,0.136994,0.575641,4960
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1,176.0,0.202983,0.550482,7843
214116,INTP,'So many questions when i do these things. I ...,0,0,0,1,21.0,0.105284,0.522723,5752
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1,60.0,0.117591,0.543356,9470


In [7]:
import nltk
from nltk import word_tokenize,sent_tokenize,ne_chunk


def split_tokens(text):
    ''' Split text into tokens.'''
    text = text.replace('/',' ')
    tokens = nltk.word_tokenize(text)
    return tokens
    
def POS_tagging(text):
    ''' Generate Part of speech tagging of the text. '''
    POSofText = nltk.tag.pos_tag(text)
    return POSofText

In [8]:
#should be done without stop words cleaned

noun = ['NN', 'NNS', 'NNP', 'NNPS']
adj = ['JJ', 'JJR', 'JJS']
prep = ['IN']
det = ['DT']
pron = ['PRP', 'PRP$', 'WP', 'WP$']
verb = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
adverb = ['RB', 'RBR', 'RBS']
interject = ['UH']

#grammar = [noun, adj, prep, pron, verb, adverb, interject]
#formality = 0.5(freq_noun + freq_adj + freq_prep + freq_article) - 0.5(freq_pron + freq_verb + freq_adverb + freq_interjection) + 50

num_noun, num_adj, num_prep, num_det, num_pron, num_verb, num_adverb, num_interject = [],[],[],[],[],[],[],[]

for index, row in raw_df.iterrows():
    count_noun, count_adj, count_prep, count_det, count_pron, count_verb, count_adverb, count_interject = 0,0,0,0,0,0,0,0
    text = row['Text']
    tokens = split_tokens(text)
    postags = POS_tagging(tokens)
    
    for word in postags:
        if word[1] in noun:
            count_noun += 1
        if word[1] in adj:
            count_adj += 1
        if word[1] in prep:
            count_prep += 1
        if word[1] in det:
            count_det += 1
        if word[1] in pron:
            count_pron += 1
        if word[1] in verb:
            count_verb += 1
        if word[1] in adverb:
            count_adverb += 1
        if word[1] in interject:
            count_interject += 1

    num_noun.append(count_noun)
    num_adj.append(count_adj)
    num_prep.append(count_prep)
    num_det.append(count_det)
    num_pron.append(count_pron)
    num_verb.append(count_verb)
    num_adverb.append(count_adverb)
    num_interject.append(count_interject)
    
raw_df['num_noun'] = num_noun
raw_df['num_adj'] = num_adj
raw_df['num_prep'] = num_prep
raw_df['num_det'] = num_det
raw_df['num_pron'] = num_pron
raw_df['num_verb'] = num_verb
raw_df['num_adverb'] = num_adverb
raw_df['num_interject'] = num_interject
raw_df

<ipython-input-8-5c23af6915aa>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['num_noun'] = num_noun
<ipython-input-8-5c23af6915aa>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['num_adj'] = num_adj
<ipython-input-8-5c23af6915aa>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Afinn Score,Polarity,Subjectivity,Length,num_noun,num_adj,num_prep,num_det,num_pron,num_verb,num_adverb,num_interject
0,INFP,Meme<3,1,0,0,0,3.0,0.000000,0.000000,7,1,0,0,0,0,1,0,0
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0,2.0,0.500000,0.888889,83,6,1,1,1,2,2,2,0
2,INTP,MemeENFP Avatar,1,0,0,0,0.0,0.000000,0.000000,16,2,0,0,0,0,0,0,0
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0,0.0,0.300000,0.300000,32,4,0,1,0,0,0,0,0
4,INTJ,StereotypesINFP 🦋,1,0,0,0,0.0,0.000000,0.000000,18,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214114,ISFP,httpswww.youtube.comwatch?v=t8edHB_h908|||IxFP...,0,0,0,1,21.0,0.136994,0.575641,4960,191,74,93,69,95,185,64,1
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1,176.0,0.202983,0.550482,7843,314,91,116,91,201,329,112,0
214116,INTP,'So many questions when i do these things. I ...,0,0,0,1,21.0,0.105284,0.522723,5752,256,69,88,80,125,210,61,1
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1,60.0,0.117591,0.543356,9470,323,113,218,137,253,398,183,3


In [9]:
def get_all_lowercase(string):
    """
    binary indicator for whether sentence is all in lowercase;
    1 for true, 0 for false
    """
    return int(string.islower())

def get_all_uppercase(string):
    """
    binary indicator for whether sentence is all in lowercase;
    1 for true, 0 for false
    """
    return int(string.isupper())

def get_uppercase_num(string):
    """
    return number of words in ALL CAPS
    """
    return len([w for w in word_tokenize(string) if w.isupper()])

def get_case(string):
    """
    binary indicator for whether the first word is capitalized.
    """
    return int(string[0].isupper())

In [10]:
def get_contractions(string):
    """
    return the number of contractions in the sentence, normalized by length
    """
    # prepare

    tokens = word_tokenize(string.lower())
    length = len(tokens)
    if length == 0:
        return 0

    # 1. number of contractions, norm by length
    cont_count = 0
    for w in tokens:
        if "\'" in w and len(w) > 1:
            cont_count += 1
    return round(cont_count/length, 2)

get_contractions("My name's Barack Obama")

0.2

In [11]:
def get_punctuation_number(string):
    """
    punctuation Number of ‘?’, ‘...’, and ‘!’ in the sentence.
    """
    punct_number = 0
    tokens = word_tokenize(string)

    for w in tokens:
        if w in ["?", "...", "!"]:
            punct_number += 1
    return punct_number

get_punctuation_number("Why are you calling me?!! AFHKSHFERGOF ERHEK894G4WE" ) #AFHKSHFERGOF readable, ERHEK894G4WE unreadable

3

In [12]:
def get_readable_words(text):
    """
    return readability Length of the sentence, in words and characters; Flesch-Kincaid Grade Level score.
    """
    try:
        blob = TextBlob(text)
        results = readability.getmeasures(text, lang='en')
        return len(blob.words)
    except ValueError:
        return 0
    
get_readable_words("Why are you calling me? AFHKSHFERGOF ERHEK894G4WE" ) #AFHKSHFERGOF readable, ERHEK894G4WE unreadable

7

In [13]:
HEDGE_WORDS = "largely generally often rarely sometimes frequently occasionally seldom usually most several some almost practically apparently virtually basically approximately roughly somewhat somehow partially actually like something someone somebody somewhere think thinks thought believe believed believes consider considers considered assume assumes assumed understand understands understood find found finds appear appears appeared seem seems seemed suppose supposes supposed guess guesses guessed estimate estimates estimated speculate speculates speculated suggest suggests suggested may could should might surely probably likely maybe perhaps unsure probable unlikely possibly possible read say says looks like look like don't know necessarily kind of much bunch couple few little really and all that and so forth et cetera in my mind in my opinion their impression my impression in my understanding my thinking is my understanding is in my view if i'm understanding you correctly something or other so far at least  about around can effectively evidently fairly hopefully in general mainly more or less mostly overall presumably  pretty quite clearly quite rather sort of supposedly  tend appear to be doubt be sure indicate will must would certainly definitely clearly conceivably certain definite clear assumption possibility probability  many almost never improbable always rare consistent with doubtful suggestive diagnostic inconclusive apparent alleged allege a bit presumable".split()
FIRST_PERSON_PRONOUNS = ['I', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours', 'my', 'mine', 'our', 'ours']
THIRD_PERSON_PRONOUNS = ['he', 'she', 'it', 'one', 'they', 'him', 'her', 'it', 'one', 'them', 'his', 'hers', 'theirs',
                         'himself', 'herself', 'itself', 'oneself', 'themselves']

def get_count(pronouns, blob):
    count = 0
    for pronoun in pronouns:
        count += blob.words.count(pronoun)
    return count

def get_hedge(text):
    blob = TextBlob(text)
    length = len(blob.words)
    if length == 0:
        return 0
    return get_count(HEDGE_WORDS, blob) / length

def get_firstperson(text):
    blob = TextBlob(text)
    length = len(blob.words)
    if length == 0:
        return 0
    return get_count(FIRST_PERSON_PRONOUNS, blob) / length

def get_thirdperson(text):
    blob = TextBlob(text)
    length = len(blob.words)
    if length == 0:
        return 0
    return get_count(THIRD_PERSON_PRONOUNS, blob) / length

get_hedge("It is proven to be largely true.")
get_firstperson("I don't like that")

0.2

In [14]:
raw_df['lowercase'] = [get_all_lowercase(text) for text in raw_df.Text]
raw_df['uppercase'] = [get_all_uppercase(text) for text in raw_df.Text]
raw_df['uppercase_num'] = [get_uppercase_num(text) for text in raw_df.Text]
raw_df['proper cap'] = [get_case(text) for text in raw_df.Text]

<ipython-input-14-db376faa7ab4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['lowercase'] = [get_all_lowercase(text) for text in raw_df.Text]
<ipython-input-14-db376faa7ab4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['uppercase'] = [get_all_uppercase(text) for text in raw_df.Text]
<ipython-input-14-db376faa7ab4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [15]:
raw_df['contractions_num'] = [get_contractions(text) for text in raw_df.Text]
raw_df['emotionalpunctuations_num'] = [get_punctuation_number(text) for text in raw_df.Text]
raw_df['readable_num'] = [get_readable_words(text) for text in raw_df.Text]

<ipython-input-15-4f0c37e1f7cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['contractions_num'] = [get_contractions(text) for text in raw_df.Text]
<ipython-input-15-4f0c37e1f7cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['emotionalpunctuations_num'] = [get_punctuation_number(text) for text in raw_df.Text]
<ipython-input-15-4f0c37e1f7cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [16]:
raw_df.to_csv(r'C:\Users\School\Desktop\4222\Project MBTI\All Features on Raw Data Part 1.csv', index = False)

In [17]:
raw_df = pd.read_excel('BT4222_Combined_dataset_Version 2.xlsx')
raw_df
# basic cleaning 
raw_df['Text'] = [str(text).replace(':', '',) for text in raw_df.Text]
raw_df['Text'] = [str(text).replace('/', '',) for text in raw_df.Text]
raw_df = raw_df[raw_df['Text']!= ""]

In [18]:
raw_df['hedge_perc'] = [get_hedge(text) for text in raw_df.Text]
raw_df['firstperson_perc'] = [get_firstperson(text) for text in raw_df.Text]
raw_df['thirdperson_perc'] = [get_thirdperson(text) for text in raw_df.Text]

raw_df

,Tag,Text,Reddit,Twitter,Typology,Kaggle,hedge_perc,firstperson_perc,thirdperson_perc
0,INFP,Meme<3,1,0,0,0,0.000000,0.000000,0.000000
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1,0,0,0,1.000000,0.187500,0.000000
2,INTP,MemeENFP Avatar,1,0,0,0,0.000000,0.000000,0.000000
3,ENTP,MemeFour Distinct Flavors of NT,1,0,0,0,0.400000,0.000000,0.000000
4,INTJ,StereotypesINFP 🦋,1,0,0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
214114,ISFP,httpswww.youtube.comwatch?v=t8edHB_h908|||IxFP...,0,0,0,1,0.439483,0.056404,0.056404
214115,ENFP,'So...if this thread already exists someplace ...,0,0,0,1,0.510504,0.111345,0.055322
214116,INTP,'So many questions when i do these things. I ...,0,0,0,1,0.397990,0.050251,0.060302
214117,INFP,'I am very conflicted right now when it comes ...,0,0,0,1,0.464088,0.109392,0.045856


In [26]:
raw_df.to_csv(r'C:\Users\School\Desktop\4222\Project MBTI\All Features on Raw Data Part 2.csv', index = False)

In [3]:
raw_df = pd.read_excel('BT4222_Combined_dataset_Version 2.xlsx')
raw_df
# basic cleaning 
raw_df['Text'] = [str(text).replace(':', '',) for text in raw_df.Text]
raw_df['Text'] = [str(text).replace('/', '',) for text in raw_df.Text]
raw_df = raw_df[raw_df['Text']!= ""]

In [22]:
def get_readability(text):
    results = readability.getmeasures(text, lang='en')
    #print(results['readability grades']['FleschReadingEase'])
    return results

#get_readability("Because of you." )

In [5]:
read_dict = get_readability("Why are you calling me?" )

measures_name = []
for department in read_dict:
    for measures in read_dict[department]:
        measures_name.append(measures)
        
readability_df = pd.DataFrame(columns = measures_name)

for index, row in raw_df.iterrows():
    measures_list = []
    text = row.Text
    try:
        read_dict = get_readability(text)
        for department in read_dict:
            for measures in read_dict[department]:
                measures_list.append(read_dict[department][measures])
    except:
        read_dict = get_readability('null')
        for department in read_dict:
            for measures in read_dict[department]:
                measures_list.append(None)

    readability_df.loc[len(readability_df)] = measures_list

final_df = pd.concat([raw_df,readability_df], axis=1)
final_df

,Tag,Text,Reddit,Twitter,Typology,Kaggle,Kincaid,ARI,Coleman-Liau,FleschReadingEase,...,conjunction,pronoun,preposition,nominalization,pronoun,interrogative,article,subordination,conjunction,preposition
0,INFP,Meme<3,1.0,0.0,0.0,0.0,-8.91,-8.655,-15.894816,162.505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ENFJ,"MemeIncorrect Quote? Not so sure. Just me, try...",1.0,0.0,0.0,0.0,4.42,5.852,6.922085,90.090,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,INTP,MemeENFP Avatar,1.0,0.0,0.0,0.0,8.79,12.540,10.564513,35.605,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENTP,MemeFour Distinct Flavors of NT,1.0,0.0,0.0,0.0,5.24,6.504,10.032935,66.400,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,INTJ,StereotypesINFP 🦋,1.0,0.0,0.0,0.0,32.00,49.720,42.809681,-132.580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214114,ISFP,httpswww.youtube.comwatch?v=t8edHB_h908|||IxFP...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214115,ENFP,'So...if this thread already exists someplace ...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214116,INTP,'So many questions when i do these things. I ...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214117,INFP,'I am very conflicted right now when it comes ...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
final_df.to_csv(r'C:\Users\School\Desktop\4222\Project MBTI\All Features on Raw Data Part 3.csv', index = False)